In [1]:
import pandas as pd
import yaml
from functions import combine_industry_codes 

pd.set_option('display.max_columns', None)

with open('../Settings.yaml', 'r') as file:
    Setting = yaml.safe_load(file)

Data = []

for year in range(Setting['Start_Year'], Setting['End_Year'] + 1):
    if year <= 1399 :
        file_name = f"Y{year}_10AndMore.xlsx"
        file_path = f"{Setting['Raw_Path']}/{file_name}"
        T08 = pd.read_excel(file_path, sheet_name='T08', skiprows=1)
        T08 = T08.rename(columns={
            'سال':'Year',
            'سطح کد فعالیت':'Industry_Category_Code',
            'کد فعالیت':'Industry_Code',
            ' فعالیت':'Industry_Name',
            ' جمع': 'Output_Services_Not_Industrial_Value',
            'حق العمل فروش      ' : 'Sale.Commission',
            '   اجاره ساختمان      ' : 'Rent.Building',
            '   اجاره ماشین آلات ' : 'Rent.Machine' ,
            '  ارتباطات و مخابرات  ' : 'Communication' , 
            'حمل و نقل' : 'Transportaion' ,
            'خسارت دریافتی ' : 'Damage' ,
            'خدمات حسابرسی و حقوقی     ' : 'Audit_Legal' ,
            '   خدمات آموزشی' : 'Education' ,
            '   تحقیقات و آزمایشگاه' : 'Research',
            ' تبلیغات، آگهی،نمایشگاه و مطبوعات' : 'Advertisement',
            ' سایر' : 'ETC'
        })
        T08 = T08[T08['Year'] != 0]
        Data.append(T08)

    
    elif year == 1400 :
        file_name = f"Y{year}_10AndMore.xlsx"
        file_path = f"{Setting['Raw_Path']}/{file_name}"
        T08 = pd.read_excel(file_path, sheet_name='T08', skiprows=1)
        T08 = T08.rename(columns={
            'سال':'Year',
            'سطح کد فعالیت':'Industry_Category_Code',
            'کد فعالیت':'Industry_Code',
            ' فعالیت':'Industry_Name',
            ' جمع': 'Output_Services_Not_Industrial_Value',
            'حق‌العمل فروش      ' : 'Sale.Commission',
            '   اجاره ساختمان      ' : 'Rent.Building',
            '   اجاره ماشین‌آلات ' : 'Rent.Machine' ,
            '  ارتباطات و مخابرات  ' : 'Communication' , 
            'حمل‌ونقل' : 'Transportaion' ,
            'خسارت دریافتی ' : 'Damage' ,
            'خدمات حسابرسی و حقوقی     ' : 'Audit_Legal' ,
            '   خدمات آموزشی' : 'Education' ,
            '   تحقیقات و آزمایشگاه' : 'Research',
            ' تبلیغات، آگهی،نمایشگاه و مطبوعات' : 'Advertisement',
            ' سایر' : 'ETC'
        })
        T08 = T08[T08['Year'] != 0]
        Data.append(T08)
    
    elif year == 1401 :
        file_name = f"Y{year}_10AndMore.xlsx"
        file_path = f"{Setting['Raw_Path']}/{file_name}"
        T08 = pd.read_excel(file_path, sheet_name='T08', skiprows=2)
        T08 = T08.rename(columns={
            'سال':'Year',
            'سطح کد فعالیت':'Industry_Category_Code',
            'کد فعالیت':'Industry_Code',
            ' فعالیت':'Industry_Name',
            ' جمع': 'Output_Services_Not_Industrial_Value',
            'حق‌العمل فروش      ' : 'Sale.Commission',
            '   اجاره ساختمان      ' : 'Rent.Building',
            '   اجاره ماشین‌آلات ' : 'Rent.Machine' ,
            '  ارتباطات و مخابرات  ' : 'Communication' , 
            'حمل‌ونقل' : 'Transportaion' ,
            'خسارت دریافتی ' : 'Damage' ,
            'خدمات حسابرسی و حقوقی     ' : 'Audit_Legal' ,
            '   خدمات آموزشی' : 'Education' ,
            '   تحقیقات و آزمایشگاه' : 'Research',
            ' تبلیغات، آگهی،نمایشگاه و مطبوعات' : 'Advertisement',
            ' سایر' : 'ETC'
        })
        T08 = T08[T08['Year'] != 0]
        Data.append(T08) 
    
Dataset = pd.concat(Data, ignore_index=True)
Dataset.dropna(subset=['Year'], inplace=True)
Dataset = Dataset[Dataset['Industry_Category_Code']!=0]
Dataset['Industry_Code'] = Dataset['Industry_Code'].str.replace('-', '_', regex=False)

In [2]:
Dataset = combine_industry_codes(Dataset, ['1101'], '1101_1102')
Dataset = combine_industry_codes(Dataset, ['267', '268'], '267_268')
Dataset = combine_industry_codes(Dataset, ['2670', '2680'], '2670_2680')
Dataset = combine_industry_codes(Dataset, ['2731', '2732','2733','2732_2733'], '2731_2732_2733')
Dataset = combine_industry_codes(Dataset, ['2818','2819'], '2818_2819')
Dataset = combine_industry_codes(Dataset, ['3011','3012'], '3011_3012')
Dataset = combine_industry_codes(Dataset, ['3211','3212'], '3211_3212')
Dataset = combine_industry_codes(Dataset, ['329','322'], '329_322')
Dataset = combine_industry_codes(Dataset, ['3290','3220'], '3290_3220')
Dataset = combine_industry_codes(Dataset, ['331','332'], '331_332')
Dataset = combine_industry_codes(Dataset, ['3311','3312','3313','3314','3319','3320','3312_3320','3313_3314','3311_3313_3319_3320'], '3311_3312_3313_3314_3319_3320')

In [3]:
Dataset.drop(columns=['Industry_Category_Code','Industry_Name'],inplace=True)
input_file = f'{Setting['Workshop_Codes']}'
Workshop_Codes = pd.read_excel(input_file)

In [4]:
Dataset = pd.merge(Dataset, Workshop_Codes, on='Industry_Code', how='left')
cols = list(Dataset.columns)
cols.remove('Industry_Category_Code')
cols.remove('Industry_Name')
cols.insert(1, 'Industry_Category_Code') 
cols.insert(3, 'Industry_Name')       
Dataset = Dataset[cols]

In [5]:
output_file_name = '07_WorkShop_Output_Not_Industrial_Service_By_Activity.xlsx'
Dataset.to_excel(f"{Setting['Output_Path']}/{output_file_name}", index=False)